In [1]:
#| default_exp utils

# Aggregation/Visualization Utils

The `HierarchicalForecast` package contains utility functions to wrangle and visualize 
hierarchical series datasets. The `aggregate` function of the module allows you to create
a hierarchy from categorical variables representing the structure levels, returning also
the aggregation contraints matrix $\mathbf{S}_{[a][b]}$.

In [2]:
#| export
import sys
import timeit
from itertools import chain
from typing import Callable, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mycolorpy import colorlist as mcp

from sklearn.preprocessing import OneHotEncoder

In [3]:
#| export
plt.rcParams['font.family'] = 'serif'

In [4]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import add_docs, show_doc

In [5]:
#| exporti
class CodeTimer:
    def __init__(self, name=None, verbose=True):
        self.name = " '"  + name + "'" if name else ''
        self.verbose = verbose

    def __enter__(self):
        self.start = timeit.default_timer()

    def __exit__(self, exc_type, exc_value, traceback):
        self.took = (timeit.default_timer() - self.start)
        if self.verbose:
            print('Code block' + self.name + \
                  ' took:\t{0:.5f}'.format(self.took) + ' seconds')

In [6]:
#| exporti
def _to_summing_matrix(S_df: pd.DataFrame):
    """Transforms the DataFrame `df` of hierarchies to a summing matrix S."""
    categories = [S_df[col].unique() for col in S_df.columns]
    cat_sizes = [len(cats) for cats in categories]
    idx_bottom = np.argmax(cat_sizes)
    cats_bottom = categories[idx_bottom]
    encoder = OneHotEncoder(categories=categories, sparse=False, dtype=np.float32)
    S = encoder.fit_transform(S_df).T
    S = pd.DataFrame(S, index=chain(*categories), columns=cats_bottom)
    tags = dict(zip(S_df.columns, categories))
    return S, tags

In [7]:
#| exporti
def _to_summing_dataframe(Y_bottom_df: pd.DataFrame,
                          spec: List[List[str]]):
    """ Create Summing dataframe

    This function receives the `Y_bottom_df` and a list of
    hierarchical levels `spec`, and computes the aggregation
    constraints dataframe `S_df`, it enriches `Y_bottom_df`
    with a bottom level 'unique_id'.

    **Parameters:**<br>
    `Y_bottom_df`: pd.DataFrame with columns `[levels, 'ds', 'y']` and levels in spec.<br>
    `spec`: `spec`: List of levels. Each element of the list contains a list of columns of `df` to aggregate.<br>

    **Returns:**<br>
    `Y_bottom_df, S_df, tags`: tuple with hierarchically structured series 
    `Y_bottom_df` ($\mathbf{y}_{[b]}$), summing dataframe `S_df`, and hierarchical 
    aggregation indexes `tags`.
    """
    #------------------------- Efficient Wrangling -------------------------#
    # Keep unique levels, preserving first aparison order
    all_levels = list(chain.from_iterable(spec))
    all_levels = [*dict.fromkeys(all_levels)]

    # Efficiently create hierarchical labels
    S_df = Y_bottom_df[all_levels].copy()
    S_df = S_df.drop_duplicates()
    
    max_len_idx = np.argmax([len(hier) for hier in spec])
    bottom_levels = spec[max_len_idx]
    for level_idx, levels in enumerate(spec):
        if levels == bottom_levels:
            # Enrich Y_bottom with unique_id
            S_df['unique_id'] = S_df[levels].agg('/'.join, axis=1)
            Y_bottom_df['unique_id'] = Y_bottom_df[levels].agg('/'.join, axis=1)
        else:
            S_df[f'Level_{[level_idx]}'] = S_df[levels].agg('/'.join, axis=1)

    S_df = S_df.drop(all_levels, axis=1)

    #------------------------------- Encoding ------------------------------#
    # One hot encode only aggregate levels
    # TODO: option to only operate with sparse matrices
    bottom_ids = list(S_df.unique_id)
    del S_df['unique_id']
    categories = [S_df[col].unique() for col in S_df.columns]
    tags = dict(zip(S_df.columns, categories))
    encoder = OneHotEncoder(categories=categories,
                            sparse=False, dtype=np.float32)
    S = encoder.fit_transform(S_df).T
    S = np.concatenate([S, np.eye(len(bottom_ids))], axis=0)
    S_df = pd.DataFrame(S, columns=bottom_ids,
                        index=list(chain(*categories))+bottom_ids)

    # Match index ordering of S_df and Y_bottom_df
    Y_bottom_df.unique_id = Y_bottom_df.unique_id.astype('category')
    Y_bottom_df.unique_id = Y_bottom_df.unique_id.cat.set_categories(S_df.columns)
    return Y_bottom_df, S_df, tags

In [8]:
#| exporti
def numpy_balance(*arrs):
    """
    Fast NumPy implementation of balance function.
    The function creates all the interactions between
    the NumPy arrays provided.

    **Parameters:**<br>
    `arrs`: NumPy arrays.<br>

    **Returns:**<br>
    `out`: NumPy array.<br>
    """
    N = len(arrs)
    out =  np.transpose(np.meshgrid(*arrs, indexing='ij'),
                        np.roll(np.arange(N + 1), -1)).reshape(-1, N)
    return out

# <span style="color:DarkBlue"> Aggregate Function </span>

In [9]:
#| exporti
def aggregate_old(df: pd.DataFrame,
                  spec: List[List[str]],
                  agg_fn: Callable = np.sum):
    """ Utils Aggregation Function.

    Aggregates bottom level series contained in the pd.DataFrame `df` according 
    to levels defined in the `spec` list applying the `agg_fn` (sum, mean).

    **Parameters:**<br>
    `df`: pd.DataFrame with columns `['ds', 'y']` and columns to aggregate.<br>
    `spec`: List of levels. Each element of the list contains a list of columns of `df` to aggregate.<br>
    `agg_fn`: Function used to aggregate `'y'`.<br>

    **Returns:**<br>
    `Y_df, S, tags`: tuple with hierarchically structured series `Y_df` ($\mathbf{y}_{[a,b]}$),
    summing matrix `S`, and hierarchical aggregation indexes `tags`.<br>
    """
    max_len_idx = np.argmax([len(hier) for hier in spec])
    bottom_comb = spec[max_len_idx]
    df_hiers = []
    for hier in spec:
        df_hier = df.groupby(hier + ['ds'])['y'].apply(agg_fn).reset_index()
        df_hier['unique_id'] = df_hier[hier].agg('/'.join, axis=1)
        if hier == bottom_comb:
            bottom_hier = df_hier['unique_id'].unique()
        df_hiers.append(df_hier)
    df_hiers = pd.concat(df_hiers)
    S_df = df_hiers[['unique_id'] + bottom_comb].drop_duplicates().reset_index(drop=True)
    S_df = S_df.set_index('unique_id')
    S_df = S_df.fillna('agg')
    hiers_cols = []
    for hier in spec:
        hier_col = '/'.join(hier) 
        S_df[hier_col] = S_df[hier].agg('/'.join, axis=1)
        hiers_cols.append(hier_col)
    Y_df = df_hiers[['unique_id', 'ds', 'y']].set_index('unique_id')
    
    # Aggregations constraints S definition
    S, tags = _to_summing_matrix(S_df.loc[bottom_hier, hiers_cols])
    return Y_df, S, tags

In [10]:
#| export
def aggregate(Y_bottom_df: pd.DataFrame,
              spec: List[List[str]],
              is_balanced: bool=False):
    """ Utils Aggregation Function.

    Aggregates bottom level series contained in the pd.DataFrame `df` according 
    to levels defined in the `spec` list applying the `agg_fn` (sum, mean).

    **Parameters:**<br>
    `Y_bottom_df`: pd.DataFrame with columns `['ds', 'y']` and columns to aggregate.<br>
    `spec`: List of levels. Each element of the list contains a list of columns of `df` to aggregate.<br>
    `is_balanced`: bool=False, wether `Y_bottom_df` is balanced, if not we balance.<br>

    **Returns:**<br>
    `Y_df, S_df, tags`: tuple with hierarchically structured series `Y_df` ($\mathbf{y}_{[a,b]}$),
    summing dataframe `S_df`, and hierarchical aggregation indexes `tags`.
    """
    #------------------------- Efficient Wrangling -------------------------#
    # Y_bottom_df's unique_id enrichment, and constraints S_df
    Y_bottom_df, S_df, tags = _to_summing_dataframe(Y_bottom_df=Y_bottom_df, 
                                                    spec=spec)

    # Create balanced/sorted dataset for efficient aggregation (nan=0)
    # balanced = for all unique_ids all date stamps are present
    if not is_balanced:
        dates         = Y_bottom_df['ds'].unique()
        balanced_prod = numpy_balance(S_df.columns, dates)
        balanced_df   = pd.DataFrame(balanced_prod, columns=['unique_id', 'ds'])
        balanced_df['ds'] = balanced_df['ds'].astype(Y_bottom_df['ds'].dtype)

        Y_bottom_df.set_index(['unique_id', 'ds'], inplace=True)
        balanced_df.set_index(['unique_id', 'ds'], inplace=True)
        balanced_df   = balanced_df.merge(Y_bottom_df[['y']],
                                          how='left', left_on=['unique_id', 'ds'],
                                          right_index=True).reset_index()
        balanced_df['y'].fillna(0, inplace=True)
        Y_bottom_df.reset_index(inplace=True)
    else:
        balanced_df = Y_bottom_df.copy()

    #------------------------- Efficient Aggregation -------------------------#
    n_agg = S_df.shape[0] - S_df.shape[1]
    Agg = S_df.values[:n_agg, :]
    y_bottom = balanced_df.y.values.reshape(len(S_df.columns), len(dates))
    y_agg = Agg @ y_bottom

    # Create long format hierarchical dataframe
    y_agg = y_agg.flatten()
    y_bottom = y_bottom.flatten()
    Y_df = pd.DataFrame(dict(
                unique_id = np.repeat(S_df.index, len(dates)),
                ds = np.tile(dates, len(S_df.index)),
                y = np.concatenate([y_agg, y_bottom], axis=0)))
    return Y_df, S_df, tags

In [11]:
#| hide
ds = pd.date_range(start='2000-01-01', end='2000-08-01', freq='MS')
y_base = np.arange(1,9)
r1 = y_base * (10**1)
r2 = y_base * (10**1)
r3 = y_base * (10**2)
r4 = y_base * (10**2)

ys = np.concatenate([r1, r2, r3, r4])
ds = np.tile(ds, 4)
unique_ids = ['r1'] * 8 + ['r2'] * 8 + ['r3'] * 8 + ['r4'] * 8
top_level = 'Australia'
middle_level = ['State1'] * 16 + ['State2'] * 16
bottom_level = unique_ids

Y_bottom_df = dict(ds=ds,
                 top_level=top_level, 
                 middle_level=middle_level, 
                 bottom_level=bottom_level,
                 y=ys)
Y_bottom_df = pd.DataFrame(Y_bottom_df)
Y_bottom_df.groupby('bottom_level').head(2)

hierarchy_levels = [['top_level'],
                    ['top_level', 'middle_level'],
                    ['top_level', 'middle_level', 'bottom_level']]

# Test equality under sorted dataframes
old_Y_df, old_S_df, old_tags = aggregate_old(Y_bottom_df, hierarchy_levels)
Y_df, S_df, tags = aggregate(Y_bottom_df, hierarchy_levels)

np.all(np.isclose(Y_df.y.values, old_Y_df.y.values, atol=1e-8))
np.all(np.isclose(S_df.to_numpy(), old_S_df.to_numpy(), atol=1e-8))

True

In [12]:
show_doc(aggregate, title_level=3)

---

[source](https://github.com/Nixtla/hierarchicalforecast/tree/main/blob/main/hierarchicalforecast/utils.py#L34){target="_blank" style="float:right; font-size:smaller"}

### aggregate

>      aggregate (Y_bottom_df:pandas.core.frame.DataFrame, spec:List[List[str]],
>                 is_balanced:bool=False)

Utils Aggregation Function.

Aggregates bottom level series contained in the pd.DataFrame `df` according 
to levels defined in the `spec` list applying the `agg_fn` (sum, mean).

**Parameters:**<br>
`Y_bottom_df`: pd.DataFrame with columns `['ds', 'y']` and columns to aggregate.<br>
`spec`: List of levels. Each element of the list contains a list of columns of `df` to aggregate.<br>
`is_balanced`: bool=False, wether `Y_bottom_df` is balanced, if not we balance.<br>

**Returns:**<br>
`Y_df, S_df, tags`: tuple with hierarchically structured series `Y_df` ($\mathbf{y}_{[a,b]}$),
summing dataframe `S_df`, and hierarchical aggregation indexes `tags`.

In [13]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
hiers_grouped = [['Country'],
                 ['Country', 'State'], 
                 ['Country', 'Purpose'], 
                 ['Country', 'State', 'Region'], 
                 ['Country', 'State', 'Purpose'], 
                 ['Country', 'State', 'Region', 'Purpose']]

hier_df, S_df, tags = aggregate_old(df=df, spec=hiers_grouped)
test_eq(len(hier_df), 34_000)
test_eq(hier_df.index.nunique(), 425)
test_eq(S_df.shape, (425, 304))
test_eq(hier_df.index.unique(), S_df.index)
test_eq(len(tags), len(hiers_grouped))

In [14]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
hiers_grouped = [['Country'],
                 ['Country', 'State'], 
                 ['Country', 'Purpose'], 
                 ['Country', 'State', 'Region'], 
                 ['Country', 'State', 'Purpose'], 
                 ['Country', 'State', 'Region', 'Purpose']]

hier_df, S_df, tags = aggregate(Y_bottom_df=df, spec=hiers_grouped)
test_eq(len(hier_df), 34_000)
test_eq(hier_df.index.nunique(), 425)
test_eq(S_df.shape, (425, 304))
test_eq(hier_df.index.unique(), S_df.index)
test_eq(len(tags), len(hiers_grouped))

AssertionError: ==:
34000
425

In [ ]:
df

# <span style="color:DarkBlue"> Hierarchical Visualization </span>

In [ ]:
#| export
class HierarchicalPlot:
    """ Hierarchical Plot

    This class contains a collection of matplotlib visualization methods, suited for small
    to medium sized hierarchical series.

    **Parameters:**<br>
    `S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate function](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
    `tags`: np.ndarray, with hierarchical aggregation indexes, where 
        each key is a level and its value contains tags associated to that level.<br><br>
    """
    def __init__(self,
                 S: pd.DataFrame,
                 tags: Dict[str, np.ndarray]):
        self.S = S
        self.tags = tags

    def plot_summing_matrix(self):
        """ Summation Constraints plot
        
        This method simply plots the hierarchical aggregation
        constraints matrix $\mathbf{S}$.
        """
        plt.figure(num=1, figsize=(4, 6), dpi=80, facecolor='w')
        plt.spy(self.S)
        plt.show()
        plt.close()

    def plot_series(self,
                    series: str,
                    Y_df: Optional[pd.DataFrame] = None,
                    models: Optional[List[str]] = None,
                    level: Optional[List[int]] = None):
        """ Single Series plot

        **Parameters:**<br>
        `series`: str, string identifying the `'unique_id'` any-level series to plot.<br>
        `Y_df`: pd.DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns `['unique_id', 'ds', 'y']`, it may have `'models'`.<br>
        `models`: List[str], string identifying filtering model columns.
        `level`: float list 0-100, confidence levels for prediction intervals available in `Y_df`.<br>

        **Returns:**<br>
        Single series plot with filtered models and prediction interval level.<br><br>
        """
        if series not in self.S.index:
            raise Exception(f'time series {series} not found')
        fig, ax = plt.subplots(1, 1, figsize = (20, 7))
        df_plot = Y_df.loc[series].set_index('ds')
        cols = models if models is not None else df_plot.columns
        cols_wo_levels = [col for col in cols if ('lo' not in col and 'hi' not in col)]
        cmap = mcp.gen_color('tab10', 10)[:len(cols_wo_levels)]
        cmap_dict = dict(zip(cols_wo_levels, cmap))
        df_plot[cols_wo_levels].plot(ax=ax, linewidth=2, color=cmap)
        if level is not None:
            for lv in level:
                for col in cols_wo_levels:
                    if col == 'y':
                        # we dont need intervals
                        # for the actual value
                        continue
                    if f'{col}-lo-{lv}' not in df_plot.columns:
                        # if model
                        # doesnt have levels
                        continue
                    ax.fill_between(
                        df_plot.index, 
                        df_plot[f'{col}-lo-{lv}'], 
                        df_plot[f'{col}-hi-{lv}'],
                        alpha=-lv/50 + 2,
                        color=cmap_dict[col],
                        label=f'{col}_level_{lv}'
                    )
        ax.set_title(f'{series} Forecast', fontsize=22)
        ax.set_xlabel('Timestamp [t]', fontsize=20)
        ax.legend(prop={'size': 15})
        ax.grid()
        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(20)
                    
    def plot_hierarchically_linked_series(self,
                                          bottom_series: str,
                                          Y_df: Optional[pd.DataFrame] = None,
                                          models: Optional[List[str]] = None,
                                          level: Optional[List[int]] = None):
        """ Hierarchically Linked Series plot

        **Parameters:**<br>
        `bottom_series`: str, string identifying the `'unique_id'` bottom-level series to plot.<br>
        `Y_df`: pd.DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns ['unique_id', 'ds', 'y'] and models. <br>
        `models`: List[str], string identifying filtering model columns.
        `level`: float list 0-100, confidence levels for prediction intervals available in `Y_df`.<br>

        **Returns:**<br>
        Collection of hierarchilly linked series plots associated with the `bottom_series`
        and filtered models and prediction interval level.<br><br>
        """
        if bottom_series not in self.S.columns:
            raise Exception(f'bottom time series {bottom_series} not found')
        linked_series = self.S[bottom_series].loc[lambda x: x == 1.].index
        fig, axs = plt.subplots(len(linked_series), 1, figsize=(20, 2 * len(linked_series)))
        cols = models if models is not None else Y_df.drop(['ds'], axis=1)
        cols_wo_levels = [col for col in cols if ('lo' not in col and 'hi' not in col)]
        cmap = mcp.gen_color('tab10', 10)[:len(cols_wo_levels)]
        cmap_dict = dict(zip(cols_wo_levels, cmap))
        for idx, series in enumerate(linked_series):
            df_plot = Y_df.loc[[series]].set_index('ds')
            df_plot[cols_wo_levels].plot(ax=axs[idx], linewidth=2, color=cmap)
            if level is not None:
                for lv in level:
                    for col in cols_wo_levels:
                        if col == 'y':
                            # we dont need intervals
                            # for the actual value
                            continue
                        if f'{col}-lo-{lv}' not in df_plot.columns:
                            # if model
                            # doesnt have levels
                            continue
                        axs[idx].fill_between(
                            df_plot.index, 
                            df_plot[f'{col}-lo-{lv}'], 
                            df_plot[f'{col}-hi-{lv}'],
                            alpha=-lv/50 + 2,
                            color=cmap_dict[col],
                            label=f'{col}_level_{lv}'
                        )
            axs[idx].set_title(f'{series}', fontsize=10)
            axs[idx].grid()
            axs[idx].get_xaxis().label.set_visible(False)
            axs[idx].legend().set_visible(False)
            for label in (axs[idx].get_xticklabels() + axs[idx].get_yticklabels()):
                label.set_fontsize(10)
        plt.subplots_adjust(hspace=0.4)
        handles, labels = axs[0].get_legend_handles_labels()
        kwargs = dict(loc='lower center', 
                      prop={'size': 10}, 
                      bbox_to_anchor=(0, 0.05, 1, 1))
        if sys.version_info.minor > 7:
            kwargs['ncols'] = np.max([2, np.ceil(len(labels) / 2)])
        fig.legend(handles, labels, **kwargs)

    def plot_hierarchical_predictions_gap(self,
                                          Y_df: pd.DataFrame,
                                          models: Optional[List[str]] = None,
                                          xlabel: Optional=None,
                                          ylabel: Optional=None,
                                          ):
        """ Hierarchically Predictions Gap plot

        **Parameters:**<br>
        `Y_df`: pd.DataFrame, hierarchically structured series ($\mathbf{y}_{[a,b]}$). 
                It contains columns ['unique_id', 'ds', 'y'] and models. <br>
        `models`: List[str], string identifying filtering model columns.
        `xlabel`: str, string for the plot's x axis label.
        `ylable`: str, string for the plot's y axis label.

        **Returns:**<br>
        Plots of aggregated predictions at different levels of the hierarchical structure.
        The aggregation is performed according to the tag levels see 
        [aggregate function](https://nixtla.github.io/hierarchicalforecast/utils.html).<br><br>
        """
        # Parse predictions dataframe
        horizon_dates = Y_df['ds'].unique()
        cols = models if models is not None else Y_df.drop(['ds', 'y'], axis=1).columns
        
        # Plot predictions across tag levels
        fig, ax = plt.subplots(figsize=(8, 5))
        
        if 'y' in Y_df.columns:
            y_plot = Y_df[Y_df.index =="['Total']"].y.values
            plt.plot(horizon_dates, y_plot, label='True')

        ys = []
        for tag in self.tags:
            y_plot = sum([Y_df[cols].loc[Y_df.index == idx].values \
                          for idx in self.tags[tag]])
            plt.plot(horizon_dates, y_plot, label=f'Level: {tag}')
            
            ys.append(y_plot[:,None])

        plt.title('Predictions Accumulated Difference')
        if ylabel is not None:
            plt.ylabel(ylabel)
        if xlabel is not None:
            plt.xlabel(xlabel)

        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
show_doc(HierarchicalPlot, title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_summing_matrix, 
         name='plot_summing_matrix', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_series, 
         name='plot_series', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_hierarchically_linked_series, 
         name='plot_hierarchically_linked_series', title_level=3)

In [ ]:
show_doc(HierarchicalPlot.plot_hierarchical_predictions_gap,
         name='plot_hierarchical_predictions_gap', title_level=3)

In [ ]:
#| hide
hplots = HierarchicalPlot(S=S, tags=tags)
hplots.plot_summing_matrix()

In [ ]:
#| hide
hier_df['Model'] = hier_df['y'] * 1.1
hier_df['Model-lo-80'] = hier_df['Model'] - 0.1 * hier_df['Model']
hier_df['Model-hi-80'] = hier_df['Model'] + 0.1 * hier_df['Model']
hplots.plot_series(
    series='Australia', 
    Y_df=hier_df,
    level=[80, 90]
)

In [ ]:
#| hide
hplots.plot_series(series='Australia', 
                   Y_df=hier_df)

In [ ]:
#| hide
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df,
    level=[80, 90]
)

In [ ]:
#| hide
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df,
)

In [ ]:
#| hide
# test series with just one value
hplots.plot_hierarchically_linked_series(
    bottom_series='Australia/Western Australia/Experience Perth/Visiting', 
    Y_df=hier_df.groupby('unique_id').tail(1),
)

In [ ]:
#| eval: false
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, ETS, Naive
from datasetsforecast.hierarchical import HierarchicalData

Y_df, S, tags = HierarchicalData.load('./data', 'Labour')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

Y_test_df  = Y_df.groupby('unique_id').tail(24)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df  = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

fcst = StatsForecast(
    df=Y_train_df, 
    #models=[AutoARIMA(season_length=12), Naive()], 
    models=[ETS(season_length=12, model='AAZ')],
    freq='MS', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(h=24)

# Plot prediction difference of different aggregation
# Levels Country, Country/Region, Country/Gender/Region ...
hplots = HierarchicalPlot(S=S, tags=tags)

hplots.plot_hierarchical_predictions_gap(
    Y_df=Y_hat_df, models='ETS',
    xlabel='Month', ylabel='Predictions',
)